Jan Nowak 268357

Schemat algorytmu dopasowania (pseudokod):

    function NeedlemanWunsch(sequence1, sequence2, match, mismatch, gap):
    // Inicjalizacja macierzy wynikowej
    Initialize matrix wynik with dimensions (len(sequence1) + 1) x (len(sequence2) + 1)
    
    // Inicjalizacja pierwszego wiersza i kolumny macierzy wynikowej
    for i from 0 to len(sequence1):
        wynik[i][0] = i * gap
    for j from 0 to len(sequence2):
        wynik[0][j] = j * gap
    
    // Wypełnienie macierzy wynikowej
    for i from 1 to len(sequence1):
        for j from 1 to len(sequence2):
            // Obliczanie punktacji dla różnych możliwości dopasowania
            match_score = match if sequence1[i - 1] == sequence2[j - 1] else mismatch
            U = wynik[i - 1][j] + gap
            L = wynik[i][j - 1] + gap
            D = wynik[i - 1][j - 1] + match_score
            
            // Wybór najlepszego wyniku
            wynik[i][j] = max(U, L, D)
    
    // Śledzenie ścieżki w macierzy wynikowej i odtwarzanie dopasowanych sekwencji
    aligned_seq1 = ""
    aligned_seq2 = ""
    i = len(sequence1)
    j = len(sequence2)
    while i > 0 and j > 0:
        if wynik[i][j] == wynik[i - 1][j - 1] + match:
            // Dopasowanie
            aligned_seq1 = sequence1[i - 1] + aligned_seq1
            aligned_seq2 = sequence2[j - 1] + aligned_seq2
            i = i - 1
            j = j - 1
        else if wynik[i][j] == wynik[i - 1][j] + gap:
            // Przerwa w sekwencji 1
            aligned_seq1 = sequence1[i - 1] + aligned_seq1
            aligned_seq2 = "_" + aligned_seq2
            i = i - 1
        else:
            // Przerwa w sekwencji 2
            aligned_seq1 = "_" + aligned_seq1
            aligned_seq2 = sequence2[j - 1] + aligned_seq2
            j = j - 1
    
    // Zwrócenie dopasowanych sekwencji
    return aligned_seq1, aligned_seq2

    


    function SmithWaterman(sequence1, sequence2, match, mismatch, gap):
    // Inicjalizacja macierzy wynikowej
    Initialize matrix wynik with dimensions (len(sequence1) + 1) x (len(sequence2) + 1)
    
    // Inicjalizacja maksymalnej wartości wyniku i jej pozycji
    max_score = 0
    max_i = 0
    max_j = 0
    
    // Wypełnienie macierzy wynikowej
    for i from 1 to len(sequence1):
        for j from 1 to len(sequence2):
            // Obliczanie punktacji dla różnych możliwości dopasowania
            match_score = match if sequence1[i - 1] == sequence2[j - 1] else mismatch
            upper = max(wynik[i - 1][j] + gap, 0)
            left = max(wynik[i][j - 1] + gap, 0)
            diagonal = max(wynik[i - 1][j - 1] + match_score, 0)
            
            // Aktualizacja wyniku i pozycji maksymalnego wyniku
            wynik[i][j] = max(upper, left, diagonal)
            if wynik[i][j] > max_score:
                max_score = wynik[i][j]
                max_i = i
                max_j = j
    
    // Śledzenie ścieżki w macierzy wynikowej i odtwarzanie dopasowanych sekwencji
    aligned_seq1 = ""
    aligned_seq2 = ""
    i = max_i
    j = max_j
    while i > 0 and j > 0 and wynik[i][j] > 0:
        if wynik[i][j] == wynik[i - 1][j - 1] + match:
            // Dopasowanie
            aligned_seq1 = sequence1[i - 1] + aligned_seq1
            aligned_seq2 = sequence2[j - 1] + aligned_seq2
            i = i - 1
            j = j - 1
        else if wynik[i][j] == wynik[i - 1][j] + gap:
            // Przerwa w sekwencji 1
            aligned_seq1 = sequence1[i - 1] + aligned_seq1
            aligned_seq2 = "_" + aligned_seq2
            i = i - 1
        else:
            // Przerwa w sekwencji 2
            aligned_seq1 = "_" + aligned_seq1
            aligned_seq2 = sequence2[j - 1] + aligned_seq2
            j = j - 1
    
    // Zwrócenie dopasowanych sekwencji
    return aligned_seq1, aligned_seq2


Oszacowanie złożoności obliczeniowej:
Zakładając, że długość sekwencji 1 wynosi n, a długość sekwencji 2 wynosi m:
operacje na macierzy wynikowej wymaga czasu O(n * m).

Instrukcja dla  dla obu przypadków:

  1  Przygotuj dwie sekwencje do dopasowania.
  
  2  Określ parametry programu: punktację za dopasowanie, karę za niedopasowanie oraz karę za przerwę.
  
  3  Uruchom algorytm Needleman-Wunsch, przekazując jako parametry sekwencje oraz ustalone wcześniej parametry.
  
  4  Algorytm zwróci dopasowane sekwencje oraz macierz wsparcia.
  
  5  Wyniki zostaną wyświetlone oraz zapisane do pliku tekstowego, w tym parametry programu, wynik dopasowania, długość dopasowania oraz procent identycznych pozycji i przerw.
  
  6 Wyświtlenie ścierzki jest realizowane przy pomocy osobnej funkcji

In [4]:
from app.UI.View import start_menu, display_sequence_input_prompt, display_sequence_retrieval_prompt, display_error
from app.Service.MainService import seq_constructor, get_uniprot_sequence, read_sequence_from_file


def start():
    while True:
        choice = start_menu()
        if choice == "1":
            name = display_sequence_input_prompt()
            seq = read_sequence_from_file(name)
            if seq:
                return seq
        elif choice == "2":
            uniprot_id = display_sequence_retrieval_prompt()
            seq = get_uniprot_sequence(uniprot_id)
            if seq:
                return seq
        else:
            display_error("Błędna opcja")


def main():
    from app.Classes.Sequence import Sequence
    from app.Classes.SequenceAlignment import SequenceAlignment

    # Tworzymy obiekty sekwencji
    seq1 = Sequence("ACGTAGC", "seq1_id")
    seq2 = Sequence("ACGTCGC", "seq2_id")

    # Tworzymy obiekt klasy SequenceAlignment
    alignment = SequenceAlignment(seq1, seq2)

    # Parametry do algorytmu
    match_score = 2
    mismatch_score = -1
    gap_penalty = -2

    # Przeprowadzamy lokalne dopasowanie za pomocą algorytmu Smitha-Watermana
    aligned_seq1, aligned_seq2, support_matrix = alignment.smith_waterman(match_score, mismatch_score, gap_penalty)

    # Wyświetlamy rezultaty
    print("Aligned Sequence 1:", aligned_seq1)
    print("Aligned Sequence 2:", aligned_seq2)

    # Wyświetlamy graficzną reprezentację macierzy z ścieżką
    alignment.sequence_path(support_matrix, "smith_path")
    print("#######################")
    # Przeprowadzamy globalne dopasowanie za pomocą algorytmu Needleman-Wunsch
    aligned_seq1_1, aligned_seq2_1, support_matrix_1 = alignment.needleman_wunsch(match_score, mismatch_score, gap_penalty)

    # Wyświetlamy rezultaty
    print("Aligned Sequence 1:", aligned_seq1_1)
    print("Aligned Sequence 2:", aligned_seq2_1)

    # Wyświetlamy graficzną reprezentację macierzy z ścieżką
    alignment.sequence_path(support_matrix_1, "neddleman_path")

if __name__ == "__main__":
    main()


In [ ]:
import numpy as np
from app.Classes.Sequence import Sequence
import matplotlib.pyplot as plt
"""
SequenceAlignment class contain methods to align and compare sequences.
    @:param Sequence: Sequence 1 to be compared
    @:param Sequence: Sequence 2 to be compared
"""


class SequenceAlignment:
    def __init__(self, sequence1, sequence2):
        self.seq1 = sequence1.get_sequence()
        self.seq2 = sequence2.get_sequence()
        self.seq1Id = sequence1.get_id()
        self.seq2Id = sequence2.get_id()
        self.seq1Name = sequence1.get_name()
        self.seq2Name = sequence2.get_name()
        self.array = self._array()

    def _array(self):
        len_seq1 = len(self.seq1)
        len_seq2 = len(self.seq2)
        return np.zeros((len_seq1, len_seq2), dtype=int)

    def _needleman_wunsch_support(self, match, mismatch, gap):
        high, width = self.array.shape
        wynik = np.zeros((high + 1, width + 1))
        support_matrix = np.zeros((high, width))

        for i in range(1, high + 1):
            wynik[i][0] = wynik[i-1][0] + gap

        for i in range(1, width + 1):
            wynik[0][i] = wynik[0][i-1] + gap

        for i in range(1, high + 1):
            for j in range(1, width + 1):
                match_score = match if self.seq1[i - 1] == self.seq2[j - 1] else mismatch
                upper = wynik[i - 1, j] + gap
                left = wynik[i, j - 1] + gap
                diagonal = wynik[i - 1, j - 1] + match_score
                wynik[i, j] = max(upper, left, diagonal)
                if wynik[i, j] == upper:
                    support_matrix[i - 1, j - 1] = 1
                elif wynik[i, j] == left:
                    support_matrix[i - 1, j - 1] = 2
                else:
                    support_matrix[i - 1, j - 1] = 0

        np.savetxt(r"D:\bioinformatyka\Lista 1\app\Output\needleman", wynik, fmt="%d")
        #print(wynik)
        #print("####################")
        #print(support_matrix)
        np.savetxt(r"D:\bioinformatyka\Lista 1\app\Output\needleman_support_matrix", support_matrix, fmt="%d")

        return support_matrix, wynik

    def needleman_wunsch(self, match, mismatch, gap):
        seq1 = ""
        seq2 = ""

        support_matrix, wynik = self._needleman_wunsch_support(match, mismatch, gap)
        high, width = support_matrix.shape
        i, j = high - 1, width - 1

        while i >= 0 and j >= 0:
            if support_matrix[i, j] == 0:
                seq1 += self.seq1[i]
                seq2 += self.seq2[j]
                i -= 1
                j -= 1
            elif support_matrix[i, j] == 1:
                seq1 += self.seq1[i]
                seq2 += "_"
                i -= 1
            elif support_matrix[i, j] == 2:
                seq1 += "_"
                seq2 += self.seq2[j]
                j -= 1

                # Odwróć sekwencje, aby zachować kolejność
        aligned_seq1 = seq1[::-1]
        aligned_seq2 = seq2[::-1]

        # Oblicz punktację dopasowania, długość dopasowania, procent identycznych pozycji i przerw
        alignment_score = 0
        identical_positions = 0
        gaps = 0
        for i in range(len(aligned_seq1)):
            if aligned_seq1[i] == aligned_seq2[i]:
                alignment_score += match
                identical_positions += 1
            elif aligned_seq1[i] == '_' or aligned_seq2[i] == '_':
                alignment_score += gap
                gaps += 1
            else:
                alignment_score += mismatch

        alignment_length = len(aligned_seq1)
        identical_positions_percent = (identical_positions / alignment_length) * 100
        gaps_percent = (gaps / alignment_length) * 100

        # Wyświetl parametry programu i wyniki
        print("Parametry programu dla needleman_wunsch:")
        print("Match:", match)
        print("Mismatch:", mismatch)
        print("Gap:", gap)
        print("Wynik dopasowania:", alignment_score)
        print("Długość dopasowania:", alignment_length)
        print("Procent identycznych pozycji:", identical_positions_percent)
        print("Procent przerw:", gaps_percent)

        with open(r"D:\bioinformatyka\Lista 1\app\Output\parameters_needleman.txt", 'w') as f:
            f.write("Sekwencja 1: {}\n".format(aligned_seq1))
            f.write("Sekwencja 2: {}\n".format(aligned_seq2))
            f.write("Parametry programu:\n")
            f.write("Match: {}\n".format(match))
            f.write("Mismatch: {}\n".format(mismatch))
            f.write("Gap: {}\n".format(gap))
            f.write("Wynik dopasowania: {}\n".format(alignment_score))
            f.write("Długość dopasowania: {}\n".format(alignment_length))
            f.write("Procent identycznych pozycji: {:.2f}%\n".format(identical_positions_percent))
            f.write("Procent przerw: {:.2f}%\n".format(gaps_percent))

        # Zwróć dopasowane sekwencje oraz macierz wsparcia
        return aligned_seq1, aligned_seq2, support_matrix

    def _smith_waterman_support(self, match, mismatch, gap):
        high, width = self.array.shape
        wynik = np.zeros((high + 1, width + 1))
        support_matrix = np.zeros((high, width))

        for i in range(1, high + 1):
            wynik[i][0] = 0  # W algorytmie Smitha-Watermana zaczynamy od zera w każdym wierszu i kolumnie.

        for i in range(1, width + 1):
            wynik[0][i] = 0

        max_score = 0  # Zmienna do śledzenia maksymalnej wartości wyniku w macierzy.

        for i in range(1, high + 1):
            for j in range(1, width + 1):
                match_score = match if self.seq1[i - 1] == self.seq2[j - 1] else mismatch
                upper = max(wynik[i - 1, j] + gap, 0)  # Ustawiamy na 0, jeśli wynik jest mniejszy od zera.
                left = max(wynik[i, j - 1] + gap, 0)
                diagonal = max(wynik[i - 1, j - 1] + match_score, 0)
                wynik[i, j] = max(upper, left, diagonal)
                if wynik[i, j] > max_score:  # Aktualizujemy maksymalną wartość wyniku.
                    max_score = wynik[i, j]
                    max_i, max_j = i, j

                # Wprowadzamy zmiany w obsłudze macierzy wsparcia, jeśli wynik jest ujemny.
                if wynik[i, j] <= 0:
                    support_matrix[i - 1, j - 1] = -1
                elif wynik[i, j] == upper:
                    support_matrix[i - 1, j - 1] = 1
                elif wynik[i, j] == left:
                    support_matrix[i - 1, j - 1] = 2
                else:
                    support_matrix[i - 1, j - 1] = 0

        np.savetxt(r"D:\bioinformatyka\Lista 1\app\Output\smith-wateman", wynik, fmt="%d")
        #print(wynik)
        print("####################")
        #print(support_matrix)
        np.savetxt(r"D:\bioinformatyka\Lista 1\app\Output\smith_support_matrix", support_matrix, fmt="%d")

        return support_matrix, wynik, max_i, max_j  # Zwracamy także pozycję maksymalnego wyniku.

    def smith_waterman(self, match, mismatch, gap):
        seq1 = ""
        seq2 = ""

        support_matrix, wynik, max_i, max_j = self._smith_waterman_support(match, mismatch, gap)

        # Początek ścieżki - największy wynik w macierzy wynikowej.
        i, j = max_i, max_j

        while i > 0 and j > 0 and wynik[
            i, j] > 0:  # Przerywamy, gdy dojdziemy do krawędzi macierzy lub wynik stanie się ujemny.
            if support_matrix[i - 1, j - 1] == 0:  # Diagonalna, kontynuuj dopasowanie.
                seq1 += self.seq1[i - 1]
                seq2 += self.seq2[j - 1]
                i -= 1
                j -= 1
            elif support_matrix[i - 1, j - 1] == 1:  # Górna, wstawienie przerwy w pierwszej sekwencji.
                seq1 += self.seq1[i - 1]
                seq2 += "_"
                i -= 1
            elif support_matrix[i - 1, j - 1] == 2:  # Lewa, wstawienie przerwy w drugiej sekwencji.
                seq1 += "_"
                seq2 += self.seq2[j - 1]
                j -= 1

                # Odwróć sekwencje, aby zachować kolejność
        aligned_seq1 = seq1[::-1]
        aligned_seq2 = seq2[::-1]

        # Oblicz punktację dopasowania, długość dopasowania, procent identycznych pozycji i przerw
        alignment_score = 0
        identical_positions = 0
        gaps = 0
        for i in range(len(aligned_seq1)):
            if aligned_seq1[i] == aligned_seq2[i]:
                alignment_score += match
                identical_positions += 1
            elif aligned_seq1[i] == '_' or aligned_seq2[i] == '_':
                alignment_score += gap
                gaps += 1
            else:
                alignment_score += mismatch

        alignment_length = len(aligned_seq1)
        identical_positions_percent = (identical_positions / alignment_length) * 100
        gaps_percent = (gaps / alignment_length) * 100

        # Wyświetl parametry programu i wyniki
        print("Parametry programu dla smith_waterman:")
        print("Match:", match)
        print("Mismatch:", mismatch)
        print("Gap:", gap)
        print("Wynik dopasowania:", alignment_score)
        print("Długość dopasowania:", alignment_length)
        print("Procent identycznych pozycji:", identical_positions_percent)
        print("Procent przerw:", gaps_percent)

        with open(r"D:\bioinformatyka\Lista 1\app\Output\parameters_smith.txt", 'w') as f:
            f.write("Sekwencja 1: {}\n".format(aligned_seq1))
            f.write("Sekwencja 2: {}\n".format(aligned_seq2))
            f.write("Parametry programu:\n")
            f.write("Match: {}\n".format(match))
            f.write("Mismatch: {}\n".format(mismatch))
            f.write("Gap: {}\n".format(gap))
            f.write("Wynik dopasowania: {}\n".format(alignment_score))
            f.write("Długość dopasowania: {}\n".format(alignment_length))
            f.write("Procent identycznych pozycji: {:.2f}%\n".format(identical_positions_percent))
            f.write("Procent przerw: {:.2f}%\n".format(gaps_percent))

        # Zwróć dopasowane sekwencje oraz macierz wsparcia
        return aligned_seq1, aligned_seq2, support_matrix
    def sequence_path(self, support_matrix, filename):
        plt.imshow(support_matrix, cmap='viridis', interpolation='nearest')
        plt.colorbar()

        # Dodanie ścieżki
        for i in range(support_matrix.shape[0]):
            for j in range(support_matrix.shape[1]):
                plt.text(j, i, int(support_matrix[i, j]), ha='center', va='center', color='white')

        plt.xlabel(self.seq1Id)
        plt.ylabel(self.seq2Id)
        plt.title('Macierz z ścieżką')
        plt.savefig("D:\\bioinformatyka\\Lista 1\\app\\Output\\"+filename+".png")
        plt.show()


In [ ]:
import numpy as np
"""
Sequence Class contains DNA sequence and associated metadata
"""
class Sequence:
    def __init__(self, seq, name="default", id=000000):
        self.sequence = seq
        self.length = len(seq)
        self.name = name
        self.id = id

    def __str__(self):
        return "Name="+ self.name+" Id="+ str(self.id) + " Sequence="+ self.sequence

    def get_length(self):
        return self.length
    def get_name(self):
        return self.name
    def get_id(self):
        return self.id
    def get_sequence(self):
        return self.sequence

In [ ]:
import requests
import re
from app.Classes.Sequence import Sequence
from app.UI.View import display_error


def seq_constructor(fasta_data):
    """
    Function construct Sequence object from fasta data
    :type fasta_data: string
    """
    lines = fasta_data.split('\n')

    # Sprawdź, czy sekwencja zawiera nagłówek
    if not lines[0].startswith('>'):
        print("Błąd: Brak nagłówka w sekwencji FASTA.")
        return None

    seq = ''.join(lines[1:])

    # Sprawdź, czy sekwencja zawiera nielegalne znaki
    basic_amino_acids = {'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W',
                         'Y'}

    if not set(seq).issubset(basic_amino_acids):
        print("Błąd: Sekwencja zawiera nielegalne znaki.")
        return None

    match = re.match(r">sp\|(\w+)\|(\w+)OS", lines[0])
    id = None
    name = None
    if match:
        id = match.group(1)
        name = match.group(2)

    return Sequence(seq, id, name)


def get_uniprot_sequence(uniprot_id):
    """
    Function get uniprot sequence from uniprot id
    :param uniprot_id:
    :return: Sequence object
    """
    url = f"https://www.uniprot.org/uniprot/{uniprot_id}.fasta"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Sprawdź, czy odpowiedź nie zawiera błędu HTTP
        fasta_data = response.text
        seq = seq_constructor(fasta_data)
        return seq
    except requests.exceptions.HTTPError as e:
        print(f"Wystąpił błąd HTTP: {e}")
        return None
    except Exception as e:
        print(f"Wystąpił inny błąd: {e}")
        return None


def read_sequence_from_file(name):
    try:
        with open(name, "r") as f:
            read = f.read()
            seq = seq_constructor(read)
            return seq
    except IOError as e:
        display_error(f"Wystąpił wyjątek: {e}")
        return None


In [ ]:
def start_menu():
    print("Wybierz metodę wprowadzania sekwencji: plik FASTA(1) lub ID Uniprot(2)")
    return input()


def display_error(message):
    print(message)


def display_sequence_input_prompt():
    print("Podaj nazwę pliku")
    return input()


def display_sequence_retrieval_prompt():
    print("Podaj ID Uniprot")
    return input()
